In [144]:
import numpy as np
from scipy.stats import geom, multivariate_normal

On fixe les paramètres.

In [145]:
N_sample = 1024
dim_x = 20
dim_z = 20
r = 0.6

On définit les fonctions utiles au projet.

In [146]:
def p_theta_x(theta):
    mu = theta
    covariance = 2 * np.eye(dim_x)
    return multivariate_normal(mean=mu, cov=covariance)

def p_theta_x_given_z(z):
    mu = z
    covariance = np.eye(dim_x)
    return multivariate_normal(mean=mu, cov=covariance)

def p_theta_z(theta):
    mu = theta
    covariance = np.eye(dim_z)
    return multivariate_normal(mean=mu, cov=covariance)

p = geom

def logmeanexp(data, axis=None):
    max_val = np.max(data, axis=axis)
    return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))

On simule theta et les observations. On calcule la log-vraisemblance.

In [147]:
theta_0 = multivariate_normal.rvs(mean=np.zeros(dim_x), cov=np.eye(dim_x))
X = p_theta_x(theta_0).rvs(size=N_sample)
print(X[:2])

l_true = np.mean(np.log(p_theta_x(theta_0).pdf(X)))
print(l_true)

[[-1.26983749 -0.05577781  0.4389024   0.29475434  1.55927259  0.56357757
  -1.11817003  1.00109145 -3.74712485  0.4779087  -0.92320417 -1.89663536
   3.38219555 -0.02181258 -3.07432439 -0.37898559 -0.17273254 -1.76626529
  -3.47280135  0.10789113]
 [-0.21641656 -0.76061392 -0.08936381  2.16839784 -2.19536959 -0.64158159
  -0.51402806  1.94249372  1.43107287 -0.39061573  0.43616129  3.31729389
  -0.53570075  0.9913435  -2.25512065  0.87710177  1.65191761 -0.84935175
  -3.7964765  -0.81093453]]
-35.32464786730774


On définit A et b.

In [148]:
A = 1 / 2 * np.eye(dim_z, dim_x) #size = dim_z, dim_x
b = np.mean(X, axis=0) #size = dim_z, 1

On peut donc maintenant définir l'encodeur.

In [149]:
def q_phi_z_given_x(x):
    mu = A @ x + b
    covariance = 2 / 3 * np.eye(dim_z)
    return multivariate_normal(mean=mu, cov=covariance)

On fait tourner l'algorithme décrit dans le papier pour chaque observation.

In [150]:
l_hat_ml_ss_list = []
l_hat_ml_rr_list = []
l_hat_iwae_list = []
l_hat_sumo_list = []

for x in X:
    
    K = p.rvs(r)
    size = 2 ** (K + 1)
    
    Z = q_phi_z_given_x(x).rvs(size=size)
    Z_E, Z_O = Z[::2], Z[1::2]
    
    log_weights = [np.log(p_theta_z(theta_0).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z)) for z in Z]
    log_weights_E, log_weights_O = log_weights[::2], log_weights[1::2]
    
    I_0 = np.mean(log_weights)
    l_hat_E, l_hat_O = logmeanexp(log_weights_E), logmeanexp(log_weights_O)
    l_hat_O_E = logmeanexp(log_weights)
    
    delta_K = l_hat_O_E - 0.5 * (l_hat_O + l_hat_E)
    l_hat_ml_ss_x = I_0 + delta_K / p(r).pmf(K)
    l_hat_ml_ss_list.append(l_hat_ml_ss_x)
    
    delta_k_list_rr = [logmeanexp(log_weights[:2**(k+1)]) - 0.5 * (logmeanexp(log_weights_O[:2**k]) + logmeanexp(log_weights_E[:2**k])) for k in range(K+1)]
    l_hat_ml_rr_x = I_0 + np.sum([delta_k_list_rr[k] / (1 - p(r).cdf(k-1)) for k in range(K+1)])
    l_hat_ml_rr_list.append(l_hat_ml_rr_x)

    l_hat_iwae_list.append(logmeanexp(log_weights))

    delta_k_list_sumo = [logmeanexp(log_weights[:2**(k+1)]) - logmeanexp(log_weights[:2**k]) for k in range(K+1)]
    l_hat_sumo_x = I_0 + np.sum([delta_k_list_sumo[k] / (1 - p(r).cdf(k-1)) for k in range(K+1)])
    l_hat_sumo_list.append(l_hat_sumo_x)


On calcule les estimateurs finaux et les biais empiriques.

In [151]:
l_hat_ml_ss = np.mean(l_hat_ml_ss_list)
print("estimateur ss =", l_hat_ml_ss)

l_hat_ml_rr = np.mean(l_hat_ml_rr_list)
print("estimateur rr =", l_hat_ml_rr)

l_hat_iwae = np.mean(l_hat_iwae_list)
print("estimateur iwae =", l_hat_iwae)

l_hat_sumo = np.mean(l_hat_sumo_list)
print("estimateur sumo =", l_hat_sumo)

empirical_bias_squared_ss = (l_hat_ml_ss - l_true) ** 2
print("carré du biais empirique de l'estimateur ss :", empirical_bias_squared_ss)

empirical_bias_squared_rr = (l_hat_ml_rr - l_true) ** 2
print("carré du biais empirique de l'estimateur rr :", empirical_bias_squared_rr)

empirical_bias_squared_iwae = (l_hat_iwae - l_true) ** 2
print("carré du biais empirique de l'estimateur iwae :", empirical_bias_squared_iwae)

empirical_bias_squared_sumo = (l_hat_sumo - l_true) ** 2
print("carré du biais empirique de l'estimateur sumo :", empirical_bias_squared_sumo)

estimateur ss = -36.60740824892005
estimateur rr = -35.527796481578264
estimateur iwae = -36.838967459766366
estimateur sumo = -35.181752881688894
carré du biais empirique de l'estimateur ss : 1.6454741966341668
carré du biais empirique de l'estimateur rr : 0.04126935948003446
carré du biais empirique de l'estimateur iwae : 2.2931638281040607
carré du biais empirique de l'estimateur sumo : 0.02041897691501
